In [37]:
import requests
from bs4 import BeautifulSoup

#lists needed for data storage
Influncer_username = []
Influncer_bio = []

#website_to_be_scraped
website1 = requests.get("https://influencers.feedspot.com/egyptian_instagram_influencers/")

#source_contente
source = website1.content

#souping_url
soup = BeautifulSoup(source, "lxml")

#Influncers bio and user names for top 97 influncers in egypt instagram
Influncer_namedata = soup.find_all('a', class_='ins_dhl')
Influncer_biodata = soup.find_all('p', class_='trow trow-wrap')
# Check if the list contains 97 values this means the list is correct
print(len(Influncer_namedata))
print(len(Influncer_biodata))


97
97


In [38]:
#Loop through the lists to extract the desired user name and bio
for name in Influncer_namedata:
    username = name.text
    Influncer_username.append(username)
for bio in Influncer_biodata:
    bio = bio.text
    Influncer_bio.append(bio)

Influncer_username_stripped = [username.strip() for username in Influncer_username]
# Check if the list contains 97 values this means the list is correct
print(len(Influncer_username_stripped),len(Influncer_bio))

97 97


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup

# Initialize the web driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the URL
url = 'https://inflact.com/tools/profile-analyzer/'
driver.get(url)

# Define lists to store the extracted data
post_per_day = []
posts_per_week = []
post_per_month = []
total_posts = []
Engagement_rate = []
total_followers = []
Avg_user_activity = []

# Loop through each influencer username
for username in Influncer_username_stripped:
    # Wait until the input field for username is present
    username_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="profileanalyzerform-username"]')))

    # Enter the username in the input field
    username_input.send_keys(username)

    # Wait until the search button is present and click it
    submit_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="account-form"]/div[2]/div[2]/button')))
    submit_button.click()

    # Wait for the page to load
    time.sleep(30)

    # Get the HTML source code of the page a queastion will rise here why we do this and we are not using beautifulsoap directly ?
    #for this website I need to pass the username to get search results so we use selenum to do this then scrape the data using beautiful soap for shorter codr running time
    html_source = driver.page_source

    # Pass the HTML source to Beautiful Soup for parsing
    soup = BeautifulSoup(html_source, 'html.parser')

    # Extract the desired metrics using Beautiful Soup
    #for the bar charts we have there numbers we need so we will get a list of all desired numbers in bar charts then extract the data from it
    #create a list to store the data and scrape the bar charts
    # Bar elements
    bar_chart_data = soup.find_all("div", class_="pa-chart-data-number")


    #note: we are using try except becouse sometimes the list is empty due to a private account or whatever and we don't want our code to crash
    try:
        number_of_posts_per_day = bar_chart_data[0].get_text(strip=True)
        post_per_day.append(number_of_posts_per_day)
    except IndexError:
        post_per_day.append("")

    try:
        number_of_posts_per_week = bar_chart_data[1].get_text(strip=True)
        posts_per_week.append(number_of_posts_per_week)
    except IndexError:
        posts_per_week.append("")

    try:
        number_of_posts_per_month = bar_chart_data[2].get_text(strip=True)
        post_per_month.append(number_of_posts_per_month)
    except IndexError:
        post_per_month.append("")



    # Extract the desired metrics using Beautiful Soup
    #for the boxes we have four numbers we need so we will get a list of all desired numbers in boxes then extract the data from it
    #create a list to store the data and scrape the bar charts
    bar_chart_data =[]

    box_values_data = soup.find_all("div", class_="pa-number-value")
    # Box elements
    box_elements = soup.find_all("div", class_="pa-number-value")

    #note: we are using try except becouse sometimes the list is empty due to a private account or whatever and we don't want our code to crash
    try:
        number_of_post = box_values_data[0].get_text(strip=True)
        total_posts.append(number_of_post)
    except IndexError:
        total_posts.append("")

    try:
        Engagmentrate = box_values_data[1].get_text(strip=True)
        Engagement_rate.append(Engagmentrate)
    except IndexError:
        Engagement_rate.append("")

    try:
        numberof_followers = box_values_data[2].get_text(strip=True)
        total_followers.append(numberof_followers)
    except IndexError:
        total_followers.append("")

    try:
        avg_activity = box_values_data[3].get_text(strip=True)
        Avg_user_activity.append(avg_activity)
    except IndexError:
        Avg_user_activity.append("")

    # Re-locate the input field for the next iteration
    username_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="profileanalyzerform-username"]')))

    # Clear the input field for the next username
    username_input.clear()

    time.sleep(10)

# Create a pandas DataFrame to store the extracted data
data = pd.DataFrame({
    'Username': Influncer_username_stripped,
    'Posts per Day': post_per_day,
    'Posts per Week': posts_per_week,
    'Posts per Month': post_per_month,
    'Total Posts': total_posts,
    'Engagement Rate': Engagement_rate,
    'Total Followers': total_followers,
    'Average User Activity': Avg_user_activity
})

# Print or save the data as desired
# Close the web driver
driver.quit()

In [35]:
# Create a pandas DataFrame to store the extracted data
data = pd.DataFrame({
    'Username': Influncer_username_stripped,
    'Posts per Day': post_per_day,
    'Posts per Week': posts_per_week,
    'Posts per Month': post_per_month,
    'Total Posts': total_posts,
    'Engagement Rate': Engagement_rate,
    'Total Followers': total_followers,
    'Average User Activity': Avg_user_activity,
    'Influncer_bio': Influncer_bio
})

In [42]:
#Export the data to a csv file
data.to_csv('instagram top 100 influncer.csv')

97
